In [38]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings("ignore")

In [39]:
def leisure_pretreat(dframe, turnOnAvg=0, choice_yr=None) : 

    """
    
    Ver. 5
    
    여가 관련 통계조사 전처리 함수입니다.

    -------------------------------------------------------------------------------

    1. (Ver. 2 -> Ver. 3 : '평균' 컬럼을 선택적으로 제외할 수 있게 수정)

    'turnOnAvg' 파라미터에 1을 입력하면 '평균' 컬럼을 유지합니다.

    2. (Ver. 3 -> Ver. 4 : '연도'를 입력하면 해당 연도 이상의 연도 리스트만 반환)

    'choice_yr' 파라미터에 연도를 입력하면, 해당 연도 이상의 컬럼만 반환합니다.

    3. (Ver. 4 -> Ver. 5 : 행 인덱스를 reset_index 했습니다.

    -------------------------------------------------------------------------------

    """

    year_List = list(range(2016,2022))
    year_List = [value for value in year_List if value != 2017]   # 2017년도 빼기

    if choice_yr : 
        year_List = [value for value in year_List if value >= choice_yr]

    category = ["연령별", "가구소득별"]

    leis_sep = dframe.iloc[:,:2]

    conc_list_age = []
    conc_list_inc = []

    for yr in year_List : 

        pattern = str(yr)
        df_name = "DataFrame_" + str(yr)

        globals()[df_name] = dframe.filter(like=pattern)    # 조건에 만족하는 데이터 프레임 반환
        globals()[df_name] = pd.concat([leis_sep, globals()[df_name]], axis=1)     # 왼쪽 오른쪽으로 합치기, 인덱스가 다르면 outer join
        globals()[df_name] = globals()[df_name].replace("-",0)
        globals()[df_name].columns = globals()[df_name].iloc[0,:].to_list()
        globals()[df_name] = globals()[df_name].iloc[1:,:]
        globals()[df_name].iloc[:,2:] = globals()[df_name].iloc[:,2:].astype(float)

        if turnOnAvg == 1:
            col_avg = globals()[df_name]["평균"]

        if "평균" in globals()[df_name].columns.to_list() : 
            globals()[df_name].drop("평균", axis=1, inplace=True)
            

        globals()[df_name].iloc[:,3:] = globals()[df_name].iloc[:,3:].apply(lambda x : x * globals()[df_name].iloc[:,2] / 100)  # 백분율로 표시된 값을 실제 값으로 변환
        globals()[df_name] = globals()[df_name].drop("표본수",axis=1)  # 합계 부분 삭제
        globals()[df_name].iloc[:,2:] = globals()[df_name].iloc[:,2:].astype(int)

        if turnOnAvg == 1 :
            globals()[df_name] = pd.concat([globals()[df_name], col_avg], axis=1)
            
        for cat1 in category : 

            df_name_c1 = "DataFrame_" + str(yr) + cat1
            globals()[df_name_c1] = globals()[df_name][globals()[df_name]["통계분류(1)"]==cat1].iloc[:,1:]
            globals()[df_name_c1].insert(0, "연도별", yr)
            globals()[df_name_c1].rename({"통계분류(2)":cat1},axis=1,inplace=True)

            if cat1 == category[0] : 
                conc_list_age.append(globals()[df_name_c1])
            else : 
                conc_list_inc.append(globals()[df_name_c1])

    df1 = pd.concat(conc_list_age, axis=0)
    df2 = pd.concat(conc_list_inc, axis=0)

    df1.fillna(0, axis=1, inplace=True)
    df2.fillna(0, axis=1, inplace=True)


    df1.reset_index(drop=True, inplace=True)
    df2.reset_index(drop=True, inplace=True)
    return df1, df2

# 여가 관련 자료

## <font color=#FFB6C1> 세계 행복 지수 </font>

In [40]:
happy_world = pd.read_excel("G:/내 드라이브/취미/세계행복지수_2020~2022.xlsx")
happy_world.head()

,순위,국가,행복지수
0,1,핀란드,7.804
1,2,덴마크,7.586
2,3,아이슬란드,7.530
3,4,이스라엘,7.473
4,5,네덜란드,7.403


## <font color=#FFE0C6> 행복수준 </font>

In [41]:
lev_happiness = pd.read_csv("G:/내 드라이브/취미/행복수준_20230510094831.csv", encoding="cp949")
lev_happiness.info()
lev_happiness.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   통계분류(1)  68 non-null     object
 1   통계분류(2)  68 non-null     object
 2   2016     68 non-null     object
 3   2016.1   68 non-null     object
 4   2016.2   68 non-null     object
 5   2016.3   68 non-null     object
 6   2016.4   68 non-null     object
 7   2016.5   68 non-null     object
 8   2018     68 non-null     object
 9   2018.1   68 non-null     object
 10  2018.2   68 non-null     object
 11  2018.3   68 non-null     object
 12  2018.4   68 non-null     object
 13  2018.5   68 non-null     object
 14  2019     68 non-null     object
 15  2019.1   68 non-null     object
 16  2019.2   68 non-null     object
 17  2019.3   68 non-null     object
 18  2019.4   68 non-null     object
 19  2019.5   68 non-null     object
 20  2020     68 non-null     object
 21  2020.1   68 non-null     object
 22  2020

,통계분류(1),통계분류(2),2016,2016.1,2016.2,2016.3,2016.4,2016.5,2018,2018.1,...,2020.2,2020.3,2020.4,2020.5,2021,2021.1,2021.2,2021.3,2021.4,2021.5
0,통계분류(1),통계분류(2),표본수,1-2점,3-4점,5-6점,7점 이상,평균,표본수,1-2점,...,3-4점,5-6점,7점 이상,평균,표본수,1-2점,3-4점,5-6점,7점 이상,평균
1,전체,소계,10602,0.3,2.4,26.2,71,7.1,10498,0.4,...,3.8,29.3,66.3,7,10049,0.5,3.7,29.7,66,7
2,성별,남성,5179,0.3,2.4,27.2,70,7.1,5263,0.5,...,3.5,28.2,67.5,7,4972,0.7,4,28.6,66.6,7
3,성별,여성,5423,0.3,2.4,25.2,72.1,7.1,5235,0.3,...,4.1,30.3,65.1,7,5077,0.4,3.4,30.8,65.4,6.9
4,연령별,15~19세,827,0.5,3.2,30.7,65.6,6.9,696,0.7,...,3.3,24.1,72.3,7.3,588,0.8,3.9,28.3,67,7.1


In [42]:
lev_happiness_age, lev_happiness_inc = leisure_pretreat(lev_happiness, turnOnAvg=1)
print(lev_happiness_age.head())
print("-*-*-*-*-*-*-**-**-*-*-*-*-*-*-*-")
print(lev_happiness_inc.tail())

    연도별     연령별 1-2점 3-4점 5-6점 7점 이상   평균
0  2016  15~19세    4   26  253   542  6.9
1  2016     20대    4   32  301  1137  7.2
2  2016     30대    5   29  419  1252  7.2
3  2016     40대    5   37  515  1431  7.1
4  2016     50대    7   43  539  1365  7.1
-*-*-*-*-*-*-**-**-*-*-*-*-*-*-*-
     연도별         가구소득별 1-2점 3-4점 5-6점 7점 이상   평균
30  2021  200-300만원 미만   11   58  504   846  6.8
31  2021  300-400만원 미만    5   87  551  1251  7.0
32  2021  400-500만원 미만    4   41  449  1113  7.1
33  2021     500-600만원    4   26  354   996  7.2
34  2021      600만원 이상    6   25  342  1354  7.4



## <font color="#FF5675"> 일과 여가 생활 균형 </font>

In [43]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')

In [44]:
A, B = leisure_pretreat(leisure)
A.head()

,연도별,연령별,일에 더 집중하고있다,일에 조금 더 집중하고 있다,다소 일에 집중하고 있다,균형을 이루고 있다,다소 여가에 더 집중하고있다,여가에 조금 더 집중하고 있다,여가에 더 집중하고 있다
0,2016,15~19세,15,95,218,284,159,48,4
1,2016,20대,11,116,302,583,320,135,5
2,2016,30대,34,162,392,599,339,175,5
3,2016,40대,45,195,443,678,426,197,1
4,2016,50대,29,183,404,690,422,217,7


## <font color="#FFD73C"> 평일 여가 활동_관광 활동 </font>

In [45]:
trip = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_관광활동_20230510095154.csv", encoding="cp949")
# trip.info()
# trip.head()

In [46]:
trip_age, trip_inc = leisure_pretreat(trip)

In [47]:
trip_inc.tail()

,연도별,가구소득별,매우불만족한다,불만족 한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
30,2021,200~300만원,0,0,1,51,162,223,87
31,2021,300~400만원,0,0,3,59,207,304,130
32,2021,400~500만원,0,0,0,58,217,273,109
33,2021,500~600만원,0,0,0,57,191,236,94
34,2021,600만원 이상,0,0,0,57,216,348,199


## <font color="#FFD73C"> 평일 여가 활동_문화예술 관람 활동 </font>

In [48]:
artCult = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_문화예술_관람활동_20230519170721.csv", encoding="cp949")
artCult.head()

,통계분류(1),통계분류(2),2016,2016.1,2016.2,2016.3,2016.4,2016.5,2016.6,2016.7,...,2020.6,2020.7,2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7
0,통계분류(1),통계분류(2),표본수,매우 불만족한다,불만족한다,다소 불만족한다,보통이다,다소 만족한다,만족한다,매우 만족한다,...,매우 만족한다,평균,표본수,불만족한다,다소 불만족한다,보통이다,다소 만족한다,만족한다,매우 만족한다,평균
1,전체,소계,3957,0,0.1,0.3,5.2,28.5,55.5,10.4,...,13.7,5.7,2068,0,0.2,5.6,30.8,47.4,15.9,5.7
2,성별,남성,1687,0,0.2,0.3,4.9,30.6,54.2,9.7,...,13.8,5.7,1011,-,0.4,6.1,30.1,46.5,16.9,5.7
3,성별,여성,2270,0,0.1,0.2,5.4,26.7,56.6,11,...,13.7,5.7,1057,0,-,5.1,31.5,48.3,15,5.7
4,연령별,15~19세,388,0,0,-,7,29.7,54,9.4,...,18.8,5.8,175,-,-,2.5,39.1,32.2,26.2,5.8


In [49]:
artCult_age, artCult_inc = leisure_pretreat(artCult)
print(artCult_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-*-")
print(artCult_inc.tail())


    연도별     연령별 매우 불만족한다 불만족한다 다소 불만족한다 보통이다 다소 만족한다 만족한다 매우 만족한다
0  2016  15~19세      0.0   0.0        0   27     115  209      36
1  2016     20대      0.0   3.0        1   27     260  543      93
2  2016     30대      0.0   0.0        2   42     231  449      83
3  2016     40대      0.0   1.0        3   46     249  419      79
4  2016     50대      0.0   0.0        1   26     156  315      63
-*-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우 불만족한다 불만족한다 다소 불만족한다 보통이다 다소 만족한다 만족한다 매우 만족한다
30  2021  200~300만원        0   0.0        2   12      77  125      44
31  2021  300~400만원        0   0.0        0   19      83  178      64
32  2021  400~500만원        0   0.0        0   17     130  181      35
33  2021  500~600만원        0   0.0        1   19     136  156      49
34  2021   600만원 이상        0   0.0        0   35     155  255     103


## <font color="#FFD73C"> 평일 여가 활동_사회 및 기타 </font>

In [50]:
sc_etc = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_사회_및_기타활동_20230510093021.csv", encoding="cp949")

In [51]:
sc_etc_age, sc_etc_inc = leisure_pretreat(sc_etc)
print(sc_etc_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-")
print(sc_etc_inc.tail())

    연도별     연령별 매우 불만족한다 불만족한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우 만족한다
0  2016  15~19세      0.0     2       2   42    201  364     130
1  2016     20대      1.0     0       5   66    374  689     228
2  2016     30대      1.0     1       3   97    474  749     219
3  2016     40대      0.0     1       7  111    578  865     227
4  2016     50대      0.0     0       1  103    495  861     293
-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우 불만족한다 불만족한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우 만족한다
30  2021  200~300만원      0.0     0       7  157    390  573     193
31  2021  300~400만원      0.0     0       6  195    488  749     284
32  2021  400~500만원      1.0     0       2  137    457  667     199
33  2021  500~600만원      0.0     2       5  109    383  558     216
34  2021   600만원 이상      0.0     0       1  114    518  659     293


## <font color="#FFD73C"> 평일 여가 활동_스포츠 참여 활동 </font>

In [52]:
doSports = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_스포츠_참여활동_20230510095111.csv", encoding="cp949")

In [53]:
doSports_age, doSports_inc = leisure_pretreat(doSports)
print(doSports_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-")
print(doSports_inc.tail())

    연도별     연령별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우만족한다
0  2016  15~19세     0.0      0       2   32    142  231     60
1  2016     20대     0.0      0       3   49    244  467    138
2  2016     30대     0.0      0       3   47    275  485    164
3  2016     40대     1.0      2       9   56    309  497    180
4  2016     50대     0.0      0       2   27    233  433    166
-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우만족한다
30  2021  200~300만원     0.0      0       2   44    166  253    121
31  2021  300~400만원     0.0      0       0   63    209  335    159
32  2021  400~500만원     0.0      0       0   60    217  300    141
33  2021  500~600만원     1.0      0       2   49    181  317    130
34  2021   600만원 이상     0.0      0       2   40    220  376    254


## <font color="#FFD73C"> 평일 여가 활동_스포츠 관람 활동 </font>

In [54]:
wacthSports = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_스포츠_관람활동_20230519170812.csv", encoding="cp949")

In [55]:
wacthSports_age, wacthSports_inc = leisure_pretreat(wacthSports)
print(wacthSports_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-")
print(wacthSports_inc.tail())

    연도별     연령별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우만족한다
0  2016  15~19세     0.0      0       0   46    147  150     40
1  2016     20대     0.0      0       2   69    273  388     73
2  2016     30대     1.0      0      10  125    337  384     65
3  2016     40대     3.0      0       5  133    388  429     61
4  2016     50대     2.0      0       0   81    328  424     50
-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우만족한다
30  2021  200~300만원       0      0       8   96    278  225     60
31  2021  300~400만원       0      0      11  125    323  330     92
32  2021  400~500만원       0      0       9   86    328  234     62
33  2021  500~600만원       0      0       4  101    235  189     87
34  2021   600만원 이상       0      0       7  100    311  285    115


## <font color="#FFD73C"> 평일 여가 활동_취미 오락 활동 </font>

In [56]:
games = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_취미・오락활동_20230510092915.csv", encoding="cp949")

In [57]:
games_age, games_inc = leisure_pretreat(games)
print(games_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-")
print(games_inc.tail())

    연도별     연령별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소 만족한다 만족한다 매우만족한다
0  2016  15~19세       0      0       1   42     237  370    148
1  2016     20대       0      1       1   74     453  698    208
2  2016     30대       1      0       6  112     530  806    178
3  2016     40대       0      0      11  123     556  959    214
4  2016     50대       1      0       7   77     563  948    211
-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소 만족한다 만족한다 매우만족한다
30  2021  200~300만원       0      0       2  140     398  573    200
31  2021  300~400만원       0      0       5  164     512  819    252
32  2021  400~500만원       0      0       4  112     482  696    205
33  2021  500~600만원       0      1       1  119     395  544    233
34  2021   600만원 이상       0      0       0  137     499  677    338


## <font color="#FFD73C"> 평일 여가 활동_휴식 활동 </font>

In [58]:
takeArest = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_휴식활동_20230510092941.csv", encoding="cp949")

In [59]:
takeArest_age, takeArest_inc = leisure_pretreat(takeArest)
print(takeArest_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-")
print(takeArest_inc.tail())

    연도별     연령별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우만족한다
0  2016  15~19세       0      0       5   54    241  389    101
1  2016     20대       0      0       5  113    433  708    173
2  2016     30대       0      1       8  158    566  748    173
3  2016     40대       0      1       7  173    654  899    217
4  2016     50대       0      3       3  130    606  911    267
-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 매우불만족한다 불만족 한다 다소불만족한다 보통이다 다소만족한다 만족한다 매우만족한다
30  2021  200~300만원       0      0       4  162    453  590    198
31  2021  300~400만원       0      1       5  215    564  837    233
32  2021  400~500만원       0      0       1  159    535  690    198
33  2021  500~600만원       0      0       4  133    440  594    194
34  2021   600만원 이상       0      1       1  145    551  707    302


## <font color="#C1FF6B	"> 지난 1년 동안 지속적 반복적 참여 </font>

In [60]:
rep_per_year = pd.read_csv("G:/내 드라이브/취미/_지난_1년_동안__지속적_반복적_참여_여가활동_여부_20230510095327.csv", encoding="cp949")
rep_per_year.info()
rep_per_year.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   통계분류(1)  68 non-null     object
 1   통계분류(2)  68 non-null     object
 2   2016     68 non-null     object
 3   2016.1   68 non-null     object
 4   2016.2   68 non-null     object
 5   2018     68 non-null     object
 6   2018.1   68 non-null     object
 7   2018.2   68 non-null     object
 8   2019     68 non-null     object
 9   2019.1   68 non-null     object
 10  2019.2   68 non-null     object
 11  2020     68 non-null     object
 12  2020.1   68 non-null     object
 13  2020.2   68 non-null     object
 14  2021     68 non-null     object
 15  2021.1   68 non-null     object
 16  2021.2   68 non-null     object
dtypes: object(17)
memory usage: 9.2+ KB


,통계분류(1),통계분류(2),2016,2016.1,2016.2,2018,2018.1,2018.2,2019,2019.1,2019.2,2020,2020.1,2020.2,2021,2021.1,2021.2
0,통계분류(1),통계분류(2),표본수,있음,없음,표본수,있음,없음,표본수,있음,없음,표본수,있음,없음,표본수,있음,없음
1,전체,소계,10602,47.2,52.8,10498,51,49,10060,48.9,51.1,10088,44.4,55.6,10049,35.4,64.6
2,성별,남성,5179,50.3,49.7,5263,52.2,47.8,5028,50.2,49.8,5003,48.2,51.8,4972,39.5,60.5
3,성별,여성,5423,44,56,5235,49.9,50.1,5032,47.7,52.3,5085,40.6,59.4,5077,31.4,68.6
4,연령별,15~19세,827,45.4,54.6,696,43.1,56.9,661,40.1,59.9,619,47.8,52.2,588,36.7,63.3


In [61]:
rep_per_year_age, rep_per_year_inc = leisure_pretreat(rep_per_year)
print(rep_per_year_age.head())
print("-*-*-*-*-*-*-*-*-*-*-*-*-*-*-")
print(rep_per_year_inc.tail())

    연도별     연령별    있음   없음
0  2016  15~19세   375  451
1  2016     20대   764  711
2  2016     30대   826  880
3  2016     40대  1013  977
4  2016     50대  1003  952
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별   있음    없음
30  2021  200~300만원  450   970
31  2021  300~400만원  608  1287
32  2021  400~500만원  583  1027
33  2021  500~600만원  576   803
34  2021   600만원 이상  785   944


## <font color="#00D7FF	"> 보다 나은 여가 생활 </font>

In [62]:
btr_leisure = pd.read_csv("G:/내 드라이브/취미/정책만족도_보다_나은_여가생활을_위해_관련_법규와_제도를_개선_20230510094233.csv", encoding="cp949")
btr_leisure.info()
btr_leisure.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 47 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   통계분류(1)  68 non-null     object
 1   통계분류(2)  68 non-null     object
 2   2016     68 non-null     object
 3   2016.1   68 non-null     object
 4   2016.2   68 non-null     object
 5   2016.3   68 non-null     object
 6   2016.4   68 non-null     object
 7   2016.5   68 non-null     object
 8   2016.6   68 non-null     object
 9   2016.7   68 non-null     object
 10  2016.8   68 non-null     object
 11  2018     68 non-null     object
 12  2018.1   68 non-null     object
 13  2018.2   68 non-null     object
 14  2018.3   68 non-null     object
 15  2018.4   68 non-null     object
 16  2018.5   68 non-null     object
 17  2018.6   68 non-null     object
 18  2018.7   68 non-null     object
 19  2018.8   68 non-null     object
 20  2019     68 non-null     object
 21  2019.1   68 non-null     object
 22  2019

,통계분류(1),통계분류(2),2016,2016.1,2016.2,2016.3,2016.4,2016.5,2016.6,2016.7,...,2020.8,2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8
0,통계분류(1),통계분류(2),표본수,전혀 만족하지 않는다,만족하지 않는다,다소 만족하지 않는다,보통이다,다소 만족한다,만족한다,매우 만족한다,...,평균,표본수,전혀 만족하지 않는다,만족하지 않는다,다소 만족하지 않는다,보통이다,다소 만족한다,만족한다,매우 만족한다,평균
1,전체,소계,10602,0.5,1.7,6.5,27.1,35.3,25,4,...,4.6,10049,0.2,1.3,8.3,39.9,33,14.4,3,4.6
2,성별,남성,5179,0.5,1.9,6.9,26.6,35,25.5,3.6,...,4.5,4972,0.2,1.5,8.2,39,33.6,14.3,3.2,4.6
3,성별,여성,5423,0.4,1.5,6.1,27.6,35.6,24.5,4.4,...,4.6,5077,0.2,1.1,8.3,40.8,32.4,14.4,2.8,4.6
4,연령별,15~19세,827,0.4,2.2,5.4,30.8,32.7,25.5,3.1,...,4.5,588,-,0.5,7.4,43.3,30.2,15.6,2.9,4.6


In [63]:
btr_leisure_age, btr_leisure_inc = leisure_pretreat(btr_leisure, turnOnAvg=1)
print(btr_leisure_age.head())
print("-*-*-*-*-*-*-*-*-*-*-")
print(btr_leisure_inc.tail())

    연도별     연령별 전혀 만족하지 않는다 만족하지 않는다 다소 만족하지 않는다 보통이다 다소 만족한다 만족한다 매우 만족한다  \
0  2016  15~19세           3       18          44  254     270  210      25   
1  2016     20대          13       20          92  346     560  373      66   
2  2016     30대          10       18         134  477     583  421      59   
3  2016     40대           9       35         129  543     704  505      61   
4  2016     50대           3       31         111  518     698  489     105   

     평균  
0  4.82  
1   4.9  
2  4.82  
3  4.84  
4  4.92  
-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 전혀 만족하지 않는다 만족하지 않는다 다소 만족하지 않는다 보통이다 다소 만족한다 만족한다  \
30  2021  200~300만원           4       18         107  608     466  173   
31  2021  300~400만원           1       34         144  773     637  254   
32  2021  400~500만원           1        8         140  631     546  236   
33  2021  500~600만원           1       17         128  505     460  223   
34  2021   600만원 이상           3       25         126  617     598  287   

   매

## <font color="#90AFFF"> 근로시간 단축 이후 </font>

In [64]:
cut_worktime = pd.read_csv("G:/내 드라이브/취미/근로시간_단축_이후_여가만족도_20230510094931.csv", encoding="cp949")
cut_worktime.info()
cut_worktime.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   통계분류(1)  67 non-null     object
 1   통계분류(2)  67 non-null     object
 2   2019     67 non-null     object
 3   2019.1   67 non-null     object
 4   2019.2   67 non-null     object
 5   2019.3   67 non-null     object
 6   2019.4   67 non-null     object
 7   2019.5   67 non-null     object
 8   2019.6   67 non-null     object
 9   2019.7   67 non-null     object
 10  2019.8   67 non-null     object
 11  2020     67 non-null     object
 12  2020.1   67 non-null     object
 13  2020.2   67 non-null     object
 14  2020.3   67 non-null     object
 15  2020.4   67 non-null     object
 16  2020.5   67 non-null     object
 17  2020.6   67 non-null     object
 18  2020.7   67 non-null     object
 19  2020.8   67 non-null     object
 20  2021     67 non-null     object
 21  2021.1   67 non-null     object
 22  2021

,통계분류(1),통계분류(2),2019,2019.1,2019.2,2019.3,2019.4,2019.5,2019.6,2019.7,...,2020.7,2020.8,2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7
0,통계분류(1),통계분류(2),표본수,더 낮아졌다,낮아졌다,다소 낮아졌다,변화없다,다소 높아졌다,높아졌다,더 높아졌다,...,더 높아졌다,평균,표본수,낮아졌다,다소 낮아졌다,변화없다,다소 높아졌다,높아졌다,더 높아졌다,평균
1,전체,소계,2711,0.1,0.1,1.4,46.3,36.8,13.9,1.4,...,0.8,4.4,3222,0.8,3.5,63.4,24.1,7.7,0.5,4.4
2,성별,남성,1684,-,0.1,1.4,46.8,35.4,14.6,1.7,...,0.6,4.4,1916,0.9,3.6,61.1,25.6,8.3,0.5,4.4
3,성별,여성,1027,0.2,0.2,1.4,45.4,39,12.9,0.8,...,1,4.3,1306,0.8,3.3,67,21.8,6.7,0.4,4.3
4,연령별,15~19세,8,-,-,-,100,-,-,-,...,-,4.3,6,-,-,64.5,13,22.5,-,4.6


In [65]:
cut_worktime_age, cut_worktime_inc = leisure_pretreat(cut_worktime, turnOnAvg=1, choice_yr=2019)
print(cut_worktime_age.head())
print('-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-')
print(cut_worktime_inc.tail())

    연도별     연령별 더 낮아졌다 낮아졌다 다소 낮아졌다 변화없다 다소 높아졌다 높아졌다 더 높아졌다   평균
0  2019  15~19세    0.0    0       0    8       0    0      0  4.0
1  2019     20대    0.0    0       9  220     177   72     10  4.7
2  2019     30대    0.0    0       8  313     244  113      7  4.7
3  2019     40대    0.0    2       6  318     274   91      8  4.7
4  2019     50대    1.0    0       6  244     220   70      8  4.7
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
     연도별      가구소득별 더 낮아졌다 낮아졌다 다소 낮아졌다 변화없다 다소 높아졌다 높아졌다 더 높아졌다   평균
16  2021  200~300만원      0    5      14  333     100   31      1  4.3
17  2021  300~400만원      0    5      29  361     158   56      3  4.4
18  2021  400~500만원      0    9      27  328     129   45      0  4.3
19  2021  500~600만원      0    4      15  325     135   40      3  4.4
20  2021   600만원 이상      0    1      14  477     217   59      5  4.4


## PDF 불러오기

In [28]:
import fitz
path1 = "G:/내 드라이브/취미/세계행복보고서_2023.pdf"

In [29]:
doc = fitz.open(path1)

pages = list(range(36,39))

count = 0

for page in doc : 
    
    count += 1

    if count in pages : 
        globals()['text_wrd_hap'+str(count)] = page.get_text()
        globals()['text_wrd_hap'+str(count)] = globals()['text_wrd_hap'+str(count)].split("\n")

doc.close()

In [31]:
for pg in pages : 
    length = len(globals()['text_wrd_hap'+str(pg)])
    print("리스트의 길이 :", length)
    print("-*-*-*-*-*-*-*-*-*-*-*-*-*-")
    print("리스트 앞에 4개의 원소 :", globals()['text_wrd_hap'+str(pg)][:10])
    print("-*-*-*-*-*-*-*-*-*-*-*-*-*-")
    print("끝에서 3개의 원소 :", globals()['text_wrd_hap'+str(pg)][-10:])
    print("\n\n")

리스트의 길이 : 1067
-*-*-*-*-*-*-*-*-*-*-*-*-*-
리스트 앞에 4개의 원소 : ['World Happiness Report 2023', '34', 'Figure 2.1: Ranking of Happiness based on a three-year-average 2020–2022 (Part 1)', '1', 'Finland', '7.804', '2', 'Denmark', '7.586', '3']
-*-*-*-*-*-*-*-*-*-*-*-*-*-
끝에서 3개의 원소 : ['5.840', '5.825', '5.819', '5.818', '5.763', '5.738', '5.722', '5.703', 'Average�Life�Evaluation', '']



리스트의 길이 : 1436
-*-*-*-*-*-*-*-*-*-*-*-*-*-
리스트 앞에 4개의 원소 : ['World Happiness Report 2023', '35', 'Figure 2.1: Ranking of Happiness based on a three-year-average 2020–2022 (Part 2)', '  Average Life Evaluation', '  95% confidence interval', 'Notes: Those with a * do not have survey information in 2022. Their averages are based on the 2020 and 2021 surveys.', '0', '2', '4', '6']
-*-*-*-*-*-*-*-*-*-*-*-*-*-
끝에서 3개의 원소 : ['����95%�i.c.�for�rank�89-107', '����95%�i.c.�for�rank�89-109', '����95%�i.c.�for�rank�89-119', '����95%�i.c.�for�rank�95-118', '����95%�i.c.�for�rank�97-118', '����95%�i.c.�for�rank�97-119', '

# 일과 여가 생활 균형_연습버전

In [ ]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')
# leisure.info()
# leisure.head(10)

In [ ]:
leisure_T = leisure.T   # 행열 전환
# leisure_T.head(5)

In [ ]:
leisure_T.replace("-",0,inplace=True)

In [ ]:
leis_uq = leisure.iloc[:,0].unique()
leis_uq = leis_uq.tolist()   # ndarray -> list
leis_uq = [re.sub(r"\(+.\)","",uq.replace(" ","_")) for uq in leis_uq]  # 괄호와 괄호 안 글자, 공백 제거
print(leis_uq)

In [ ]:
leisure_T.columns = leisure_T.iloc[0,:]
# leisure_T.info()

In [ ]:
columns_nospace = [re.sub(r"\(+.\)", "", col.replace(" ","_")) for col in leisure_T.columns]

In [ ]:
leisure_T.columns = columns_nospace
print(leisure_T.columns)

In [ ]:
# 통계 분류별로 데이터 프레임 분류

for cat in leis_uq : 
    globals()["leisure_{}".format(cat)] = leisure_T[cat]
    
    if type(globals()["leisure_{}".format(cat)]) == pd.core.series.Series :   # type이 시리즈라면
        globals()["leisure_{}".format(cat)] = pd.DataFrame(globals()["leisure_{}".format(cat)])   # 시리즈를 데이터프레임으로 변경
        
    globals()["leisure_{}".format(cat)].columns = globals()["leisure_{}".format(cat)].iloc[1,:]
    globals()["leisure_{}".format(cat)] = globals()["leisure_{}".format(cat)].iloc[2:,:]
    globals()["leisure_{}".format(cat)].iloc[:,1:] = globals()["leisure_{}".format(cat)].iloc[:,1:].astype(float)

In [ ]:
leis_uq2 = leis_uq[1:]
print(leis_uq2)

In [ ]:
for uq2 in leis_uq2 : 
    globals()["leis_{0}_{1}".format(uq2, "합본")] = pd.concat([leisure_통계분류, globals()["leisure_{}".format(uq2)]],axis=1)

In [ ]:
for uniq in leis_uq2 : 
    idx_List = globals()["leis_{0}_{1}".format(uniq, "합본")].index.to_list()   # 분리된 데이터프레임 인덱스 리스트로 반환
    
    for yr in year_List : 
        sep_year = []    # 정규표현식에 만족하는 인덱스를 저장할 리스트
        
        for idx in idx_List :
            find_yr = re.findall(str(yr)+"[.0-9]*",idx)     # 연도 + '.' or 숫자 1개 이상 포함돼 있는 인덱스 저장 

            if find_yr:      # 빈 리스트가 아니라면 (findall은 조건에 안 맞으면 빈 리스트 반환)
                sep_year.append(find_yr[0])

        Last_idx = len(sep_year)
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, "합본")][:][sep_year[0]:sep_year[Last_idx-1]]    # 행 인덱스로 슬라이싱, 행 인덱스로 슬라이싱할 때 콜론 뒤 값을 포함해서 슬라이싱한다.
        pattern = str(yr) + r'[.0-9]*'
        new_index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.map(lambda x: re.sub(pattern, str(yr), x))   # map 함수를 이용해 인덱스마다 정규 표현식 sub를 적용
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = new_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.astype(int)   # 인덱스를 int 타입으로
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"통계분류(2)" : "분류"}, inplace=True)   # 컬럼명 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].reset_index()   # reset_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"index" : "연도"}, inplace=True)    # 인덱스가 컬럼으로 바뀌고, 그 컬럼명을 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].set_index(["연도","분류"])   # 멀티 인덱스 설정


# 전체 -> 리스트 중에서 2016년이면 리스트에 저장 -> 리스트에 저장된 첫번째와 마지막 인덱스로 슬라이싱 -> 새로운 데이터 프레임 생성

In [ ]:
leis_17개_시도별_2016_T = leis_17개_시도별_2016.T
leis_17개_시도별_2016_T.head(10)

In [ ]:
df = leis_17개_시도별_2016_T.stack(level=0)
pd.DataFrame(df)


In [ ]:
pd.concat([leis_17개_시도별_2016, leis_가구소득별_2016],axis=1)